In [ ]:
import hashlib
from PIL import Image

# Open the image file
image_file = Image.open("/content/bwimages.jpg")

# Convert the image to bytes
image_bytes = image_file.tobytes()

# Generate the hash object using SHA-512 algorithm
hash_object = hashlib.sha512()

# Update the hash object with the image bytes
hash_object.update(image_bytes)

# Generate the hash value in hexadecimal format
hash_hex = hash_object.hexdigest()
# Print the hash value
print("SHA-512 hash value:", hash_hex)

SHA-512 hash value: e75ec59b72cf8886a936302e812e61a079876acbc1daef44c608a77de789a24cff6183031beb3e6e9148ac4c4e7dbb8563310d65ce4257460c1214aee7db7d9a


In [ ]:
#DIVIDING INTO 4 PARTS
#Our contribution
import textwrap
prekeys=textwrap.wrap(hash_hex, 32)
prekeys

['e75ec59b72cf8886a936302e812e61a0',
 '79876acbc1daef44c608a77de789a24c',
 'ff6183031beb3e6e9148ac4c4e7dbb85',
 '63310d65ce4257460c1214aee7db7d9a']

In [ ]:
keys=[]
for i in prekeys:
  dec_value = int(i, 16)
  keys.append(dec_value%0.9999)
keys

[0.04603859763960827,
 0.8555441413199993,
 0.04946458116016572,
 0.9828668749154592]

In [ ]:
!pip install cryptography

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#ELLIPTIC CURVE KEY GENERATION
import os
import hashlib
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat

# Generate a private key using P-256 curve
private_key = ec.generate_private_key(ec.SECP256R1())

# Get the corresponding public key
public_key = private_key.public_key()

# Convert the public key to bytes and hash it
public_key_bytes = public_key.public_bytes(Encoding.X962, PublicFormat.CompressedPoint)
key_hash = hashlib.sha256(public_key_bytes).digest()

In [ ]:
# Encrypting keys
en=[]
for i in keys:
  original_value = i
  value_bytes = bytes(str(original_value), 'utf-8')
  encrypted_value = bytes([value_bytes[i] ^ key_hash[i % len(key_hash)] for i in range(len(value_bytes))])
  en.append(encrypted_value)
  # Decrypt the encrypted value
  decrypted_value_bytes = bytes([encrypted_value[i] ^ key_hash[i % len(key_hash)] for i in range(len(encrypted_value))])
  decrypted_value = float(decrypted_value_bytes.decode('utf-8'))
  # Output the results
  print(en)
  print(encrypted_value)
  print(f"Original value: {original_value}")
  print(f"Encrypted value: {encrypted_value}")
  #print(f"decrypted value: {decrypted_value}")

[b'\x15\xd0Z\x06&\xd8ADS\xb7p\xe1}\x89d\x81;6\xf2']
b'\x15\xd0Z\x06&\xd8ADS\xb7p\xe1}\x89d\x81;6\xf2'
Original value: 0.04603859763960827
Encrypted value: b'\x15\xd0Z\x06&\xd8ADS\xb7p\xe1}\x89d\x81;6\xf2'
[b'\x15\xd0Z\x06&\xd8ADS\xb7p\xe1}\x89d\x81;6\xf2', b'\x15\xd0R\x07%\xddFHW\xbav\xe4\x7f\x89k\x88:7']
b'\x15\xd0R\x07%\xddFHW\xbav\xe4\x7f\x89k\x88:7'
Original value: 0.8555441413199993
Encrypted value: b'\x15\xd0R\x07%\xddFHW\xbav\xe4\x7f\x89k\x88:7'
[b'\x15\xd0Z\x06&\xd8ADS\xb7p\xe1}\x89d\x81;6\xf2', b'\x15\xd0R\x07%\xddFHW\xbav\xe4\x7f\x89k\x88:7', b'\x15\xd0Z\x06)\xdcDHS\xb6v\xe6x\x80c\x8763\xf7']
b'\x15\xd0Z\x06)\xdcDHS\xb6v\xe6x\x80c\x8763\xf7'
Original value: 0.04946458116016572
Encrypted value: b'\x15\xd0Z\x06)\xdcDHS\xb6v\xe6x\x80c\x8763\xf7'
[b'\x15\xd0Z\x06&\xd8ADS\xb7p\xe1}\x89d\x81;6\xf2', b'\x15\xd0R\x07%\xddFHW\xbav\xe4\x7f\x89k\x88:7', b'\x15\xd0Z\x06)\xdcDHS\xb6v\xe6x\x80c\x8763\xf7', b'\x15\xd0S\n"\xd0DJ^\xb9s\xee\x7f\x85f\x84:6']
b'\x15\xd0S\n"\xd0DJ^\xb9s\xee\x7f\x

In [ ]:
#GENERATING KEYSTREAMS USING LOGISTIC CHAOTIC MAP
import numpy as np
import cv2
image=cv2.imread('/content/bwimages.jpg')
flat_image = image.flatten()
height,width,c=image.shape

def logistic_map(x, r):
    """
    Generate key stream values using the logistic map.
    """
    key_stream = []
    for i in range(height):
        x = r * x * (1 - x)
        key_stream.append(int(np.round(x * height - 1))%height)
    return key_stream

# Set the initial value and parameters for the logistic map

r = 3.99

# Generate the key stream using the logistic map
key_stream1 = logistic_map(keys[0], r)
key_stream2 = logistic_map(keys[1], r)
key_stream3 = logistic_map(keys[2], r)
key_stream4 = logistic_map(keys[3], r)

# Print the first 10 values of the key stream1
print(key_stream1[:10])

[498, 1641, 2773, 286, 1030, 2623, 822, 2334, 1677, 2750]


In [ ]:
# Print the first 10 values of the key stream1
print(key_stream1[:10])
# Print the first 10 values of the key stream2
print(key_stream2[:10])

[498, 1641, 2773, 286, 1030, 2623, 822, 2334, 1677, 2750]
[1403, 2839, 29, 119, 459, 1537, 2822, 99, 385, 1331]


In [ ]:
from PIL import Image
import numpy as np

image = Image.open('/content/bwimages.jpg')
image_array = np.array(image)


In [ ]:
#GENERATING PERMUTATION SEQUENCE
#Our contribution
key = np.array(key_stream1[:5]) # replace with your float array key
num_rows = image_array.shape[0]
permutation_indices = np.random.RandomState(seed=key).permutation(num_rows)


In [ ]:
#ROW PERMUTATION
#Our contribution
permuted_image_array = image_array[permutation_indices, :]

In [ ]:
permuted_image = Image.fromarray(permuted_image_array)
permuted_image.save('permuted_image.png')

In [ ]:
from PIL import Image
import numpy as np

image = Image.open('/content/permuted_image.png')
image_array = np.array(image)

In [ ]:
#GENERATING PERMUTATION SEQUENCE
#Our contribution
key = np.array(key_stream2[:5]) # replace with your float array key
num_cols = image_array.shape[1]
permutation_indices = np.random.RandomState(seed=key).permutation(num_cols)

In [ ]:
##COLUMN PERMUTATION
#Our contribution
permuted_image_array = image_array[:,permutation_indices, :]

In [ ]:
permuted_image = Image.fromarray(permuted_image_array)
permuted_image.save('col_permuted_image.png')

In [ ]:
#DCT
import cv2
import numpy as np

# Load image
img = cv2.imread('col_permuted_image.png')

# Split image into channels
b, g, r = cv2.split(img)

# Define DCT function
def dct2(block):
    return cv2.dct(cv2.dct(block.T).T)



# Perform DCT on each channel
dct_b = dct2(b.astype(np.float32))
dct_g = dct2(g.astype(np.float32))
dct_r = dct2(r.astype(np.float32))

dct_img = cv2.merge((dct_b,dct_g,dct_r))
cv2.imwrite('dct_image.png', dct_img)

True

In [ ]:
#GENERATING SUBSTITUTION BOX
#Our contribution
import numpy as np
from PIL import Image

# Load the image data from a file
image = np.array(Image.open("dct_image.png"))

# Calculate the size of the substitution box needed
sbox_size = image.shape[0] * image.shape[1] * image.shape[2]

# Generate a random substitution box with the required size
sbox = np.arange(sbox_size, dtype=np.uint8)
np.random.shuffle(sbox)

# Reshape the substitution box to match the input image shape
sbox = sbox.reshape(image.shape)

# Generate a random key with the same shape as the input image
key = np.random.randint(256, size=image.shape).astype(np.uint8)

# Save the substitution box and key to files
np.save("sbox.npy", sbox)
arr1=np.load("sbox.npy")
print(arr1)
np.save("key.npy", key)
arr2=np.load("key.npy")
print("keys:")
print(arr2)

[[[222 244 206]
  [164 192  44]
  [211 169 151]
  ...
  [249 241  84]
  [ 23 181 254]
  [107 120  49]]

 [[178 130 151]
  [179 247 116]
  [225 241 155]
  ...
  [244 190 204]
  [ 64  52 170]
  [178 162  71]]

 [[ 68  60 252]
  [144   3 100]
  [117 253  34]
  ...
  [180  40 248]
  [224  88  73]
  [  3 207  81]]

 ...

 [[ 37 194 141]
  [216 193 204]
  [ 49 231 199]
  ...
  [118 202 163]
  [237 189  47]
  [ 85 111 203]]

 [[ 36 154  68]
  [134 253 192]
  [ 99 129 127]
  ...
  [165  17 167]
  [121 223 145]
  [ 33   8 224]]

 [[198  26 183]
  [188 173  34]
  [203 179 215]
  ...
  [105 190 142]
  [152 118 227]
  [110  93 248]]]
keys:
[[[152  96 251]
  [172 194  54]
  [ 75 179 240]
  ...
  [147  39   3]
  [144  90  48]
  [ 30 172   3]]

 [[129 115  34]
  [ 37 203 106]
  [141  83  37]
  ...
  [ 82  49  45]
  [ 73  58  33]
  [238 174  94]]

 [[ 58  90  34]
  [177  25  60]
  [ 43 204 223]
  ...
  [167 249  15]
  [120  20 157]
  [111 104  54]]

 ...

 [[230  84 203]
  [174 227 124]
  [ 66  86 163

In [ ]:
#PERFORMING BITWISE XOR
#Our contribution
imgforps = Image.open("/content/dct_image.png")
img_array = np.array(imgforps)
encrypted_array = np.bitwise_xor(img_array, arr2)
encrypted_img = Image.fromarray(encrypted_array)
encrypted_img.save("sub_encrypted_image.png")

In [ ]:
import random
from PIL import Image

# Generate public and private keys
def generate_keypair(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)
    e = random.randrange(1, phi)
    while gcd(e, phi) != 1:
        e = random.randrange(1, phi)
    d = modinv(e, phi)
    return ((n, e), (n, d))

# Greatest common divisor
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

# Modular inverse
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    return x % m

# Extended Euclidean algorithm
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

# Convert image to sequence of numbers
def image_to_numbers(image_path):
    with Image.open(image_path) as img:
        pixels = img.load()
        width, height = img.size
        numbers = []
        for y in range(height):
            for x in range(width):
                r, g, b = pixels[x, y]
                numbers.extend([r, g, b])
    return numbers, width, height

# Convert sequence of numbers to image
def numbers_to_image(numbers, width, height, image_path):
    img = Image.new('RGB', (width, height))
    pixels = img.load()
    i = 0
    for y in range(height):
        for x in range(width):
            r = numbers[i]
            g = numbers[i + 1]
            b = numbers[i + 2]
            pixels[x, y] = (r, g, b)
            i += 3
    img.save(image_path)

# Encrypt a message using a public key
def encrypt(public_key, message):
    n, e = public_key
    encrypted = [pow(char, e, n) for char in message]
    return encrypted

In [ ]:
# Example usage
p = 179
q = 2789
public_key, private_key = generate_keypair(p, q)
image_path = '/content/sub_encrypted_image.png'
numbers, width, height = image_to_numbers(image_path)
encrypted_numbers = encrypt(public_key, numbers)
numbers_to_image(encrypted_numbers, width, height, 'final_encrypted_img.png')

In [ ]:
##DECRYPTION

In [ ]:
#RSA decryption

In [ ]:
def decrypt(private_key, message):
    n, d = private_key
    decrypted = [pow(char, d, n) for char in message]
    return decrypted

In [ ]:
dimage_path = '/content/final_encrypted_img.png'
numbersd1, widthd1, heightd1 = image_to_numbers(dimage_path)
decrypted_numbers = decrypt(private_key, encrypted_numbers)
numbers_to_image(decrypted_numbers, widthd1, heightd1, 'rsa_decrypted_img.png')

In [ ]:
#pixel substitution on rsa decrypted image
#Our contribution
deimgforps = Image.open("/content/rsa_decrypted_img.png")
deimg_array = np.array(deimgforps)
decrypted_array = np.bitwise_xor(deimg_array, arr2)
decrypted_img = Image.fromarray(decrypted_array)
decrypted_img.save("decrypted_image_from_ps.png")

In [ ]:
#IDCT on decyrpted image grom pixel substitution
import cv2
import numpy as np

# Load image
img = cv2.imread('decrypted_image_from_ps.png')

# Split image into channels
b, g, r = cv2.split(img)

# Define IDCT function
def idct2(block):
    return cv2.idct(cv2.idct(block.T).T)

idct_b = idct2(dct_b).astype(np.uint8)
idct_g = idct2(dct_g).astype(np.uint8)
idct_r = idct2(dct_r).astype(np.uint8)

idct_img = cv2.merge((idct_b, idct_g, idct_r))
cv2.imwrite('idct_image.png', idct_img)

True

In [ ]:
#DECRYPTING KEYS USING ELLIPTIC CURVE CRYPTOGRAPHY
en_len = len(en)
dkeys=[]
print(en_len)
for i in range(en_len):
  a=en[i]
  decrypted_value_bytes = bytes([a[i] ^ key_hash[i % len(key_hash)] for i in range(len(a))])
  decrypted_value = float(decrypted_value_bytes.decode('utf-8'))
  dkeys.append(decrypted_value)
  # Output the results
  print(f"decrypted value: {decrypted_value}")
print(dkeys)

4
decrypted value: 0.04603859763960827
decrypted value: 0.8555441413199993
decrypted value: 0.04946458116016572
decrypted value: 0.9828668749154592
[0.04603859763960827, 0.8555441413199993, 0.04946458116016572, 0.9828668749154592]


In [ ]:
#keystream generation using logistic chaotic map

In [ ]:
d_image=cv2.imread('/content/idct_image.png')

In [ ]:
d_flat_image = d_image.flatten()
d_height,d_width,d_c=d_image.shape

def d_logistic_map(x, r):
    """
    Generate key stream values using the logistic map.
    """
    d_key_stream = []
    for i in range(d_height):
        x = r * x * (1 - x)
        d_key_stream.append(int(np.round(x * d_height - 1))%d_height)
    return d_key_stream

# Set the initial value and parameters for the logistic map

r = 3.99

# Generate the key stream using the logistic map
d_key_stream1 = d_logistic_map(dkeys[0], r)
d_key_stream2 = d_logistic_map(dkeys[1], r)
d_key_stream3 = d_logistic_map(dkeys[2], r)
d_key_stream4 = d_logistic_map(dkeys[3], r)

# Print the first 10 values of the key stream
print(d_key_stream1[:10])

[498, 1641, 2773, 286, 1030, 2623, 822, 2334, 1677, 2750]


In [ ]:
# Print the first 10 values of the key stream1 using decrypted value
print(d_key_stream1[:10])
# Print the first 10 values of the key stream2 using decrypted value
print(d_key_stream2[:10])

[498, 1641, 2773, 286, 1030, 2623, 822, 2334, 1677, 2750]
[1403, 2839, 29, 119, 459, 1537, 2822, 99, 385, 1331]


In [ ]:
# Print all the values of the key stream
print(d_key_stream1)

[498, 1641, 2773, 286, 1030, 2623, 822, 2334, 1677, 2750, 375, 1300, 2819, 111, 428, 1453, 2839, 32, 130, 497, 1640, 2774, 283, 1018, 2610, 867, 2406, 1485, 2834, 49, 195, 726, 2159, 2081, 2234, 1918, 2496, 1226, 2786, 239, 876, 2421, 1445, 2839, 30, 122, 468, 1562, 2813, 134, 511, 1676, 2751, 371, 1289, 2815, 126, 484, 1603, 2794, 206, 764, 2231, 1926, 2485, 1259, 2802, 175, 657, 2018, 2344, 1652, 2766, 311, 1108, 2701, 551, 1774, 2667, 670, 2046, 2295, 1774, 2667, 670, 2046, 2297, 1771, 2671, 659, 2022, 2337, 1668, 2755, 352, 1234, 2790, 222, 818, 2328, 1694, 2737, 420, 1430, 2840, 28, 112, 432, 1465, 2837, 37, 149, 566, 1812, 2628, 805, 2304, 1752, 2689, 596, 1881, 2546, 1072, 2667, 672, 2049, 2291, 1784, 2657, 705, 2118, 2164, 2071, 2252, 1877, 2551, 1057, 2652, 722, 2152, 2095, 2208, 1977, 2410, 1476, 2836, 43, 173, 649, 2001, 2371, 1580, 2806, 162, 612, 1919, 2496, 1228, 2786, 235, 864, 2401, 1499, 2832, 59, 234, 860, 2395, 1516, 2828, 75, 294, 1055, 2651, 729, 2165, 2069, 2255, 

In [ ]:
#reversing column permutation on image got from IDCT
#Our contribution

In [ ]:
dp_image = Image.open('/content/idct_image.png')
dp_image_array = np.array(dp_image)

In [ ]:
dkey = np.array(d_key_stream2[:5]) # replace with your float array key
num_cols1 = dp_image_array.shape[1]
re_permutation_indices = np.random.RandomState(seed=dkey).permutation(num_cols1)

In [ ]:
inverse_indices = np.argsort(re_permutation_indices)

In [ ]:
re_permuted_image_array = dp_image_array[:,inverse_indices, :]

In [ ]:
re_permuted_image = Image.fromarray(re_permuted_image_array)
re_permuted_image.save('reverse_col_permuted_image.png')

In [ ]:
#reversing row permutation
#Our contribution

In [ ]:
dp_image = Image.open('/content/reverse_col_permuted_image.png')
dp_image_array = np.array(dp_image)

In [ ]:
dkey1 = np.array(d_key_stream1[:5]) # replace with your float array key
num_rows1 = dp_image_array.shape[0]
re_permutation_indices = np.random.RandomState(seed=dkey1).permutation(num_rows1)

In [ ]:
inverse_indices = np.argsort(re_permutation_indices)

In [ ]:
re_permuted_image_array = dp_image_array[inverse_indices, :]

In [ ]:
re_permuted_image = Image.fromarray(re_permuted_image_array)
re_permuted_image.save('final_decrypted_image.png')